<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/test/notebooks/model-merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merging CodeLLMs to Create an Efficant Low-Memory Quantized Model for `whats-up-doc`

## Download and Install `mergekit`

In [1]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 2265, done.
remote: Counting objects: 100% (1354/1354), done.
remote: Compressing objects: 100% (520/520), done.
remote: Total 2265 (delta 1081), reused 947 (delta 833), pack-reused 911
Receiving objects: 100% (2265/2265), 640.50 KiB | 10.33 MiB/s, done.
Resolving deltas: 100% (1584/1584), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 13.9 MB/s

## Create the YAML Config File to Merge Models with SLERP

In [2]:
import yaml

### Write Config Script

In [3]:
# Set model name
MODEL_NAME = 'StarCodeLlama-instruct-slerp'

# Write YAML config string
yamlConfig = """
slices:
  - sources:
      - model: bigcode/starcoder2-7b
        layer_range: [0, 32]
      - model: codellama/CodeLlama-7b-Instruct-hf
        layer_range: [0, 32]
merge_method: slerp
base_model: codellama/CodeLlama-7b-Instruct-hf
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16

"""

*Note: If you were to do this locally, instead of putting in the models' card name under `model`, you would specify the path to the model you downloaded from huggingface.*

### Save Config Script

In [5]:
# Save config string as a YAML file
with open('config.yaml', 'w', encoding="utf-8") as fout:
    fout.write(yamlConfig)

## Merge Models

In [ ]:
# Merge models
!mergekit-yaml config.yaml merge \
    --copy-tokenizer \
    --allow-crimes \
    --out-shard-size 1B \
    --lazy-unpickle

config.json: 100% 646/646 [00:00<00:00, 2.67MB/s]
config.json: 100% 893/893 [00:00<00:00, 3.06MB/s]
Warmup loader cache:   0% 0/2 [00:00<?, ?it/s]
Fetching 10 files:   0% 0/10 [00:00<?, ?it/s]

special_tokens_map.json: 100% 411/411 [00:00<00:00, 1.15MB/s]


model.safetensors.index.json: 100% 25.1k/25.1k [00:00<00:00, 41.4MB/s]


tokenizer.json:   0% 0.00/1.84M [00:00<?, ?B/s]


pytorch_model.bin.index.json: 100% 23.9k/23.9k [00:00<00:00, 36.8MB/s]



model-00002-of-00002.safetensors:   0% 0.00/3.50G [00:00<?, ?B/s]



generation_config.json: 100% 116/116 [00:00<00:00, 279kB/s]

Fetching 10 files:  20% 2/10 [00:00<00:02,  2.87it/s]



model-00001-of-00002.safetensors:   0% 0.00/9.98G [00:00<?, ?B/s]




tokenizer.model:   0% 0.00/500k [00:00<?, ?B/s]





tokenizer_config.json: 100% 1.59k/1.59k [00:00<00:00, 4.91MB/s]
tokenizer.model: 100% 500k/500k [00:00<00:00, 10.9MB/s]



model-00002-of-00002.safetensors:   1% 21.0M/3.50G [00:00<00:26, 132MB/s]



model-00001-of-00002.safetensors:  